In [234]:
import pandas as pd
import os
import gc
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, minmax_scale,OneHotEncoder
from gensim.models import Word2Vec
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')


In [235]:
news_list = []
for idx, line in enumerate(open('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_news.csv', encoding='utf-8')):
    if idx == 0:
        cols = line.split('|')
    else:
        line_list = line.split('|')
        line_list = line_list[:8] + [''.join(line_list[8:]).replace('\n','')]
        news_list.append(line_list)
news_df = pd.DataFrame(news_list, columns=cols)

ent_default = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_default.csv', sep='|')
ent_fina = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_financial_indicator.csv', sep='|')
ent_info = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/2022S-T3-1st-Training/ent_info.csv', sep='|')
answer = pd.read_csv('/home/c506plus/wq/AIWIN/国泰/sample_submission/answer.csv', sep='|')


In [236]:
### 时间处理
ent_default['year'] = ent_default['acu_date'].apply(lambda x:x//10000)
ent_fina['year'] = ent_fina['report_period'].apply(lambda x:x//10000)
news_df['year'] = news_df['publishdate'].apply(lambda x:int(x)//10000)

In [237]:
#这里造个object特征减掉一年
ent_default['ent_id_year'] = ent_default['ent_id'] + '_' + (ent_default['year'] - 1).astype(str)

In [238]:
ent_fina['ent_id_year'] = ent_fina['ent_id'] + '_' + ent_fina['year'].astype(str)

In [239]:
news_df['ent_id_year'] = news_df['ent_id'] + '_' + news_df['year'].astype(str)

In [240]:
answer['ent_id_year'] = answer['ent_id'].apply(lambda x: x+'_2020')

In [241]:
del ent_fina['year'], news_df['year']

In [242]:
# 去重
ent_default_new = ent_default.drop_duplicates(subset=['ent_id_year'], keep='last')

In [243]:
ent_default_new['default_score'] = 1

In [244]:
answer['year'] = 2021

In [245]:
data = pd.concat([ent_default_new[['ent_id','ent_id_year','year','default_score']], answer], axis=0, ignore_index=True)
del ent_default_new

In [246]:
ent_ids = [i for i in answer['ent_id'].unique() if i not in ent_default[ent_default['year']==2019]['ent_id'].unique().tolist()]

In [247]:
ent_ids_df = pd.DataFrame({'ent_id':ent_ids})

In [248]:
ent_ids_df['year'] = 2019
ent_ids_df['default_score'] = 0
ent_ids_df['ent_id_year'] = ent_ids_df['ent_id'].apply(lambda x: x+'_2018')

In [249]:
data = pd.concat([data, ent_ids_df], axis=0, ignore_index=True)

In [250]:
ent_ids = [i for i in answer['ent_id'].unique() if i not in ent_default[ent_default['year']==2020]['ent_id'].unique().tolist()]
ent_ids_df = pd.DataFrame({'ent_id':ent_ids})
ent_ids_df['year'] = 2020
ent_ids_df['default_score'] = 0
ent_ids_df['ent_id_year'] = ent_ids_df['ent_id'].apply(lambda x: x+'_2019')
data = pd.concat([data, ent_ids_df], axis=0, ignore_index=True)

In [251]:
# 合并ent_info、ent_fina
print(data.shape)
ent_info_new = ent_info.drop_duplicates()
data = data.merge(ent_info_new, on=['ent_id'], how='left')

print(data.shape)

ent_fina_new = ent_fina.sort_values('report_period').drop_duplicates(subset=['ent_id_year'], keep='last')
data = data.merge(ent_fina_new, on=['ent_id','ent_id_year'], how='left')
print(data.shape)

(26926, 4)
(26926, 17)
(26926, 180)


In [252]:
# numerical_features = list(data.select_dtypes(exclude=['object']).columns)
# category_features = list(filter(lambda x: x not in numerical_fea,list(data.columns)))

In [253]:
#日期时间处理optfrom opto esdate reegap,opto这一列暂时先将其抛弃掉
for feat in['opfrom','esdate','apprdate']:
    data[feat+'_year']=pd.to_datetime(data[feat]).dt.year
    data[feat+'_month']=pd.to_datetime(data[feat]).dt.month
    data[feat+'_day']=pd.to_datetime(data[feat]).dt.day

In [254]:
data.head(1)

,ent_id,ent_id_year,year,default_score,industryphy,industryco,opfrom,opto,esdate,apprdate,regcap,enttype,entstatus,prov,city,county,is_bondissuer,report_period,s_fa_extraordinary,s_fa_deductedprofit,s_fa_grossmargin,s_fa_operateincome,s_fa_investincome,s_stmnote_finexp,s_stm_is,s_fa_ebit,s_fa_ebitda,s_fa_fcff,s_fa_fcfe,s_fa_exinterestdebt_current,s_fa_exinterestdebt_noncurrent,s_fa_interestdebt,s_fa_netdebt,s_fa_tangibleasset,s_fa_workingcapital,s_fa_networkingcapital,s_fa_investcapital,s_fa_retainedearnings,s_fa_eps_basic,s_fa_eps_diluted,s_fa_eps_diluted2,s_fa_bps,s_fa_ocfps,s_fa_grps,s_fa_orps,s_fa_surpluscapitalps,s_fa_surplusreserveps,s_fa_undistributedps,s_fa_retainedps,s_fa_cfps,s_fa_ebitps,s_fa_fcffps,s_fa_fcfeps,s_fa_netprofitmargin,s_fa_grossprofitmargin,s_fa_cogstosales,s_fa_expensetosales,s_fa_profittogr,s_fa_saleexpensetogr,s_fa_adminexpensetogr,s_fa_finaexpensetogr,s_fa_impairtogr_ttm,s_fa_gctogr,s_fa_optogr,s_fa_ebittogr,s_fa_roe,s_fa_roe_deducted,s_fa_roa2,s_fa_roa,s_fa_roic,s_fa_roe_yearly,s_fa_roa2_yearly,s_fa_roe_avg,s_fa_operateincometoebt,s_fa_investincometoebt,s_fa_nonoperateprofittoebt,s_fa_taxtoebt,s_fa_deductedprofittoprofit,s_fa_salescashintoor,s_fa_ocftoor,s_fa_ocftooperateincome,s_fa_capitalizedtoda,s_fa_debttoassets,s_fa_assetstoequity,s_fa_dupont_assetstoequity,s_fa_catoassets,s_fa_ncatoassets,s_fa_tangibleassetstoassets,s_fa_intdebttototalcap,s_fa_equitytototalcapital,s_fa_currentdebttodebt,s_fa_longdebtodebt,s_fa_current,s_fa_quick,s_fa_cashratio,s_fa_ocftoshortdebt,s_fa_debttoequity,s_fa_equitytodebt,s_fa_equitytointerestdebt,s_fa_tangibleassettodebt,s_fa_tangassettointdebt,s_fa_tangibleassettonetdebt,s_fa_ocftodebt,s_fa_ocftointerestdebt,s_fa_ocftonetdebt,s_fa_ebittointerest,s_fa_longdebttoworkingcapital,s_fa_ebitdatodebt,s_fa_turndays,s_fa_invturndays,s_fa_arturndays,s_fa_invturn,s_fa_arturn,s_fa_caturn,s_fa_faturn,s_fa_assetsturn,s_fa_roa_yearly,s_fa_dupont_roa,s_stm_bs,s_fa_prefinexpense_opprofit,s_fa_nonopprofit,s_fa_optoebt,s_fa_noptoebt,s_fa_ocftoprofit,s_fa_cashtoliqdebt,s_fa_cashtoliqdebtwithinterest,s_fa_optoliqdebt,s_fa_optodebt,s_fa_roic_yearly,s_fa_tot_faturn,s_fa_profittoop,s_qfa_operateincome,s_qfa_investincome,s_qfa_deductedprofit,s_qfa_eps,s_qfa_netprofitmargin,s_qfa_grossprofitmargin,s_qfa_expensetosales,s_qfa_profittogr,s_qfa_saleexpensetogr,s_qfa_adminexpensetogr,s_qfa_finaexpensetogr,s_qfa_impairtogr_ttm,s_qfa_gctogr,s_qfa_optogr,s_qfa_roe,s_qfa_roe_deducted,s_qfa_roa,s_qfa_operateincometoebt,s_qfa_investincometoebt,s_qfa_deductedprofittoprofit,s_qfa_salescashintoor,s_qfa_ocftosales,s_qfa_ocftoor,s_fa_yoyeps_basic,s_fa_yoyeps_diluted,s_fa_yoyocfps,s_fa_yoyop,s_fa_yoyebt,s_fa_yoynetprofit,s_fa_yoynetprofit_deducted,s_fa_yoyocf,s_fa_yoyroe,s_fa_yoybps,s_fa_yoyassets,s_fa_yoyequity,s_fa_yoy_tr,s_fa_yoy_or,s_qfa_yoygr,s_qfa_cgrgr,s_qfa_yoysales,s_qfa_cgrsales,s_qfa_yoyop,s_qfa_cgrop,s_qfa_yoyprofit,s_qfa_cgrprofit,s_qfa_yoynetprofit,s_qfa_cgrnetprofit,s_fa_yoy_equity,rd_expense,opfrom_year,opfrom_month,opfrom_day,esdate_year,esdate_month,esdate_day,apprdate_year,apprdate_month,apprdate_day
0,q00350f4fa8694d82639aec1653cf0306,q00350f4fa8694d82639aec1653cf0306_2018,2019,1,房地产业,其他房地产业,2010-05-11,2030-05-10,2010-05-11,2014-12-12,25000.0,有限责任公司（自然人投资或控股）,在营（开业）,安徽,合肥市,蜀山区,Y,20181231.0,0.0,0.0,1.534363e+09,-2.312607e+09,64689162.49,2.324603e+09,1.815677e+08,27390064.29,2.089578e+08,5.374499e+09,6.850113e+09,1.646651e+10,2.156353e+09,1.676312e+10,1.480098e+10,-1.131705e+09,-4.761203e+09,3.457175e+09,2.494674e+10,-6.144273e+08,0.0,0.0,-10.9711,10.3591,-0.0712,28.0338,28.0338,0.0,0.0,-2.4577,-2.4577,-4.0547,0.1096,21.498,27.4005,-37.0633,21.8931,78.1069,48.3651,-37.0633,6.566,7.1141,34.685,3.163,132.9974,-31.8067,0.3908,-64.9922,0.0,0.062,-5.8795,0.116,-64.9922,0.062,0.0,0.0,0.0,0.0,0.0,0.0,1.0002,-0.0025,0.0,2.3451,81.2171,5.324,10.4689,50.2319,49.7681,-2.5975,67.1956,10.3812,75.3039,24.6961,0.8213,0.328,0.2537,-0.0007,4.324,0.0732,0.1545,-0.032,-0.0675,-0.0765,-0.0005,-0.0011,-0.0012,

In [255]:
#构建交叉特征
# data['pro_city_county']=data['prov']+'_'+data['city']+'_'+data['county']
data['industryphy_yco']=data['industryphy']+'_'+data['industryco']

In [256]:
#对object特征进行label_encoder处理
for feat in ['ent_id_year','industryphy','industryco','enttype',
             'entstatus','prov','city','county','is_bondissuer','industryphy_yco']:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [258]:
#对object特征进行frequency_encoder处理
for feat in ['ent_id','ent_id_year','industryphy','industryco','enttype',
             'entstatus','prov','city','county','is_bondissuer','industryphy_yco']:
    data[feat+'_cnt']=data[feat].map(data[feat].value_counts())

In [259]:
#对连续型特征进行缺失值补全和归一化
dense_features=[
    's_fa_extraordinary','s_fa_deductedprofit','s_fa_grossmargin','s_fa_operateincome',
    's_fa_investincome','s_stmnote_finexp','s_stm_is','s_fa_ebit','s_fa_ebitda',
    's_fa_fcff','s_fa_fcfe','s_fa_exinterestdebt_current','s_fa_exinterestdebt_noncurrent',
    's_fa_interestdebt','s_fa_netdebt','s_fa_tangibleasset','s_fa_workingcapital',
    's_fa_networkingcapital','s_fa_investcapital','s_fa_retainedearnings','s_fa_eps_basic',
    's_fa_eps_diluted','s_fa_eps_diluted2','s_fa_bps','s_fa_ocfps','s_fa_grps','s_fa_orps',
    's_fa_surpluscapitalps','s_fa_surplusreserveps','s_fa_undistributedps','s_fa_retainedps',
    's_fa_cfps','s_fa_ebitps','s_fa_fcffps','s_fa_fcfeps','s_fa_netprofitmargin','s_fa_grossprofitmargin',
    's_fa_cogstosales','s_fa_expensetosales','s_fa_profittogr','s_fa_saleexpensetogr','s_fa_adminexpensetogr',
    's_fa_finaexpensetogr','s_fa_impairtogr_ttm','s_fa_gctogr','s_fa_optogr','s_fa_ebittogr','s_fa_roe',
    's_fa_roe_deducted','s_fa_roa2','s_fa_roa','s_fa_roic','s_fa_roe_yearly','s_fa_roa2_yearly',
    's_fa_roe_avg','s_fa_operateincometoebt','s_fa_investincometoebt','s_fa_nonoperateprofittoebt',
    's_fa_taxtoebt','s_fa_deductedprofittoprofit','s_fa_salescashintoor','s_fa_ocftoor','s_fa_ocftooperateincome',
    's_fa_capitalizedtoda','s_fa_debttoassets','s_fa_assetstoequity','s_fa_dupont_assetstoequity',
    's_fa_catoassets','s_fa_ncatoassets','s_fa_tangibleassetstoassets','s_fa_intdebttototalcap',
    's_fa_equitytototalcapital','s_fa_currentdebttodebt','s_fa_longdebtodebt','s_fa_current',
    's_fa_quick','s_fa_cashratio','s_fa_ocftoshortdebt','s_fa_debttoequity','s_fa_equitytodebt',
    's_fa_equitytointerestdebt','s_fa_tangibleassettodebt','s_fa_tangassettointdebt','s_fa_tangibleassettonetdebt',
    's_fa_ocftodebt','s_fa_ocftointerestdebt','s_fa_ocftonetdebt','s_fa_ebittointerest',
    's_fa_longdebttoworkingcapital','s_fa_ebitdatodebt','s_fa_turndays','s_fa_invturndays','s_fa_arturndays',
    's_fa_invturn','s_fa_arturn','s_fa_caturn','s_fa_faturn','s_fa_assetsturn','s_fa_roa_yearly',
    's_fa_dupont_roa','s_stm_bs','s_fa_prefinexpense_opprofit','s_fa_nonopprofit','s_fa_optoebt',
    's_fa_noptoebt','s_fa_ocftoprofit','s_fa_cashtoliqdebt','s_fa_cashtoliqdebtwithinterest',
    's_fa_optoliqdebt','s_fa_optodebt','s_fa_roic_yearly','s_fa_tot_faturn','s_fa_profittoop',
    's_qfa_operateincome','s_qfa_investincome','s_qfa_deductedprofit','s_qfa_eps','s_qfa_netprofitmargin',
    's_qfa_grossprofitmargin','s_qfa_expensetosales','s_qfa_profittogr','s_qfa_saleexpensetogr',
    's_qfa_adminexpensetogr','s_qfa_finaexpensetogr','s_qfa_impairtogr_ttm','s_qfa_gctogr','s_qfa_optogr',
    's_qfa_roe','s_qfa_roe_deducted','s_qfa_roa','s_qfa_operateincometoebt','s_qfa_investincometoebt',
    's_qfa_deductedprofittoprofit','s_qfa_salescashintoor','s_qfa_ocftosales','s_qfa_ocftoor','s_fa_yoyeps_basic',
    's_fa_yoyeps_diluted','s_fa_yoyocfps','s_fa_yoyop','s_fa_yoyebt','s_fa_yoynetprofit','s_fa_yoynetprofit_deducted',
    's_fa_yoyocf','s_fa_yoyroe','s_fa_yoybps','s_fa_yoyassets','s_fa_yoyequity','s_fa_yoy_tr','s_fa_yoy_or',
    's_qfa_yoygr','s_qfa_cgrgr','s_qfa_yoysales','s_qfa_cgrsales','s_qfa_yoyop','s_qfa_cgrop','s_qfa_yoyprofit',
    's_qfa_cgrprofit','s_qfa_yoynetprofit','s_qfa_cgrnetprofit','s_fa_yoy_equity','rd_expense','opfrom_year',
    'opfrom_month','opfrom_day','esdate_year','esdate_month','esdate_day','apprdate_year','apprdate_month',
    'apprdate_day','ent_id_cnt','ent_id_year_cnt','industryphy_cnt','industryco_cnt','enttype_cnt','entstatus_cnt',
    'prov_cnt','city_cnt','county_cnt','is_bondissuer_cnt', 'opfrom_year','opfrom_month', 
             'opfrom_day', 'esdate_year', 'esdate_month',
             'esdate_day', 'apprdate_year', 'apprdate_month', 'apprdate_day',
    'industryphy_yco_cnt']
data[dense_features] = data[dense_features].fillna(0, )
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [260]:
del data['opfrom']
del data['opto']
del data['esdate']
del data['apprdate']


In [261]:
data.head()

,ent_id,ent_id_year,year,default_score,industryphy,industryco,regcap,enttype,entstatus,prov,city,county,is_bondissuer,report_period,s_fa_extraordinary,s_fa_deductedprofit,s_fa_grossmargin,s_fa_operateincome,s_fa_investincome,s_stmnote_finexp,s_stm_is,s_fa_ebit,s_fa_ebitda,s_fa_fcff,s_fa_fcfe,s_fa_exinterestdebt_current,s_fa_exinterestdebt_noncurrent,s_fa_interestdebt,s_fa_netdebt,s_fa_tangibleasset,s_fa_workingcapital,s_fa_networkingcapital,s_fa_investcapital,s_fa_retainedearnings,s_fa_eps_basic,s_fa_eps_diluted,s_fa_eps_diluted2,s_fa_bps,s_fa_ocfps,s_fa_grps,s_fa_orps,s_fa_surpluscapitalps,s_fa_surplusreserveps,s_fa_undistributedps,s_fa_retainedps,s_fa_cfps,s_fa_ebitps,s_fa_fcffps,s_fa_fcfeps,s_fa_netprofitmargin,s_fa_grossprofitmargin,s_fa_cogstosales,s_fa_expensetosales,s_fa_profittogr,s_fa_saleexpensetogr,s_fa_adminexpensetogr,s_fa_finaexpensetogr,s_fa_impairtogr_ttm,s_fa_gctogr,s_fa_optogr,s_fa_ebittogr,s_fa_roe,s_fa_roe_deducted,s_fa_roa2,s_fa_roa,s_fa_roic,s_fa_roe_yearly,s_fa_roa2_yearly,s_fa_roe_avg,s_fa_operateincometoebt,s_fa_investincometoebt,s_fa_nonoperateprofittoebt,s_fa_taxtoebt,s_fa_deductedprofittoprofit,s_fa_salescashintoor,s_fa_ocftoor,s_fa_ocftooperateincome,s_fa_capitalizedtoda,s_fa_debttoassets,s_fa_assetstoequity,s_fa_dupont_assetstoequity,s_fa_catoassets,s_fa_ncatoassets,s_fa_tangibleassetstoassets,s_fa_intdebttototalcap,s_fa_equitytototalcapital,s_fa_currentdebttodebt,s_fa_longdebtodebt,s_fa_current,s_fa_quick,s_fa_cashratio,s_fa_ocftoshortdebt,s_fa_debttoequity,s_fa_equitytodebt,s_fa_equitytointerestdebt,s_fa_tangibleassettodebt,s_fa_tangassettointdebt,s_fa_tangibleassettonetdebt,s_fa_ocftodebt,s_fa_ocftointerestdebt,s_fa_ocftonetdebt,s_fa_ebittointerest,s_fa_longdebttoworkingcapital,s_fa_ebitdatodebt,s_fa_turndays,s_fa_invturndays,s_fa_arturndays,s_fa_invturn,s_fa_arturn,s_fa_caturn,s_fa_faturn,s_fa_assetsturn,s_fa_roa_yearly,s_fa_dupont_roa,s_stm_bs,s_fa_prefinexpense_opprofit,s_fa_nonopprofit,s_fa_optoebt,s_fa_noptoebt,s_fa_ocftoprofit,s_fa_cashtoliqdebt,s_fa_cashtoliqdebtwithinterest,s_fa_optoliqdebt,s_fa_optodebt,s_fa_roic_yearly,s_fa_tot_faturn,s_fa_profittoop,s_qfa_operateincome,s_qfa_investincome,s_qfa_deductedprofit,s_qfa_eps,s_qfa_netprofitmargin,s_qfa_grossprofitmargin,s_qfa_expensetosales,s_qfa_profittogr,s_qfa_saleexpensetogr,s_qfa_adminexpensetogr,s_qfa_finaexpensetogr,s_qfa_impairtogr_ttm,s_qfa_gctogr,s_qfa_optogr,s_qfa_roe,s_qfa_roe_deducted,s_qfa_roa,s_qfa_operateincometoebt,s_qfa_investincometoebt,s_qfa_deductedprofittoprofit,s_qfa_salescashintoor,s_qfa_ocftosales,s_qfa_ocftoor,s_fa_yoyeps_basic,s_fa_yoyeps_diluted,s_fa_yoyocfps,s_fa_yoyop,s_fa_yoyebt,s_fa_yoynetprofit,s_fa_yoynetprofit_deducted,s_fa_yoyocf,s_fa_yoyroe,s_fa_yoybps,s_fa_yoyassets,s_fa_yoyequity,s_fa_yoy_tr,s_fa_yoy_or,s_qfa_yoygr,s_qfa_cgrgr,s_qfa_yoysales,s_qfa_cgrsales,s_qfa_yoyop,s_qfa_cgrop,s_qfa_yoyprofit,s_qfa_cgrprofit,s_qfa_yoynetprofit,s_qfa_cgrnetprofit,s_fa_yoy_equity,rd_expense,opfrom_year,opfrom_month,opfrom_day,esdate_year,esdate_month,esdate_day,apprdate_year,apprdate_month,apprdate_day,industryphy_yco,ent_id_cnt,ent_id_year_cnt,industryphy_cnt,industryco_cnt,enttype_cnt,entstatus_cnt,prov_cnt,city_cnt,county_cnt,is_bondissuer_cnt,industryphy_yco_cnt
0,q00350f4fa8694d82639aec1653cf0306,93,2019,1,10,117,25000.0,33,1,8,56,1184,0,20181231.0,0.563578,0.150621,0.068243,0.223391,0.079164,0.147999,0.000820,0.102500,0.095773,0.454926,0.420775,0.016382,0.001276,1.580004e-03,0.015746,0.068638,0.936090,0.377159,0.002642,0.060986,0.428200,0.447040,0.103957,0.000788,0.599683,0.002891,0.002891,0.000018,8.720208e-07,0.018885,0.018885,0.460533,0.028065,0.735906,0.715416,0.687454,0.948978,0.051022,0.027559,0.720117,0.031915,0.000012,0.796565,0.386897,0.025004,0.749805,0.772449,0.794129,0.866590,0.609578,0.652282,0.975341,0.794129,0.609578,0.72473,0.996182,0.000974,0.494467,0.121585,0.905414,0.000034,0.744642,0.340494,0.027416,0.135145,0.000948,0.004713,0.502319,0.498860,0.980042,0.020106,0.739032,0.767609,0.232311,0.00

In [262]:
features = [f for f in data.columns if f not in ['ent_id','default_score']]
train = data[data.year!=2021].reset_index(drop=True)
test = data[data.year==2021].reset_index(drop=True)
x_train = train[features]
x_test = test[features]

y_train = train['default_score']

In [209]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2022
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.01,
                'seed': 2022,
                'n_jobs':-1,
                'verbose': -1,
            }

            model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=500, early_stopping_rounds=500)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            # print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
            
        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)
       
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test
    
def lgb_model(x_train, y_train, x_test):
    lgb_train, lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_train, lgb_test
    
lgb_train, lgb_test = lgb_model(x_train, y_train, x_test)

************************************ 1 ************************************
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.986732	valid_1's auc: 0.954909
[1000]	training's auc: 0.996027	valid_1's auc: 0.967353
[1500]	training's auc: 0.998496	valid_1's auc: 0.969589
[2000]	training's auc: 0.999322	valid_1's auc: 0.969608
Early stopping, best iteration is:
[1650]	training's auc: 0.998801	valid_1's auc: 0.969943
[0.9699425839053812]
************************************ 2 ************************************
Training until validation scores don't improve for 500 rounds
[500]	training's auc: 0.987009	valid_1's auc: 0.947301
[1000]	training's auc: 0.995679	valid_1's auc: 0.954533
[1500]	training's auc: 0.998402	valid_1's auc: 0.957012
[2000]	training's auc: 0.999298	valid_1's auc: 0.957807
[2500]	training's auc: 0.999673	valid_1's auc: 0.958687
Early stopping, best iteration is:
[2484]	training's auc: 0.999669	valid_1's auc: 0.958761
[0.9699425839053812

In [114]:
test['default_score'] = lgb_test
answer = pd.merge(answer[['ent_id']], test[['ent_id','default_score']], on=['ent_id'], how='left')
answer[['ent_id', 'default_score']].to_csv('4_30_2.csv', header=True, index=False, sep='|')